# Höherdimensionale Funktionen und höhergradige Polynome
Bisher wurden ausschließlich Ausgleichsgeraden für eine unabhängige und eine abhängige Variable berechnet. Es gibt aber durchaus funktionale Zusammenhänge, die mehr als eine unabhängige Variable einbeziehen oder die sich nicht linear verhalten. In diesem Notebook werden diese beiden Fälle auf Basis der Methode der kleinsten Quadrate angeschnitten.

In [ ]:
import random

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from tui_dsg.regression import draw_regression, R_squared
from tui_dsg.datasets import xyz_regression_path

## Inhaltsverzeichnis
- [Höhergradige Polynome](#Höhergradige-Polynome)
- [Höherdimensionale Funktionen](#Höherdimensionale-Funktionen)

## Höhergradige Polynome
Zunächst soll wieder ein künstlicher Datensatz erzeugt werden, der sich an einer quadratischen Funktion mit den Parametern $5$, $-2$ und $1$ orientiert. An der eingezeichneten Ausgleichsgerade lässt sich erkennen, dass die Methode der kleinsten Quadrate ohne Anpassungen keine korrekten Resultate bei derartigen Polynomen liefert.

In [ ]:
np.random.seed(1)

df_sqr = pd.DataFrame({
    'x': (random.random() * 10 - 4 for _ in range(200))
})
df_sqr['y'] = 5 - 2 * df_sqr['x'] + df_sqr['x'] ** 2
df_sqr['y'] += np.random.normal(loc=0, scale=2, size=200)

px.scatter(df_sqr, x='x', y='y', trendline='ols')

Ist allerdings der Grad des Polynoms bekannt, dem die Ausgleichsfunktion angehören soll, lässt sich die Methode der kleinsten Quadrate sehr leicht adaptieren. Im Folgenden wird das beispielhaft anhand der quadratischen Funktion veranschaulicht. Das Vorgehen lässt sich aber auch auf kubische oder noch höhergradige Polynome verallgemeinern.

Da im Beispiel von einer quadratischen Ausgleichsfunktion ausgegangen wird, wird der funktionale Zusammenhang angepasst. Um leichter verallgemeinern zu können, wird außerdem nicht mehr $\alpha$ und $\beta$ verwendet, sondern $\alpha_0$, $\alpha_1$, ... statt diesen.

$$
y = \alpha_0 + \alpha_1 * x + \alpha_2 * x^2
$$

Die Residuen und auch die Residuenfunktion lassen sich nun wie folgt formulieren:

$$
r_i = \alpha_0 + \alpha_1 * x_i + \alpha_2 * x_i^2 - y_i
$$
$$
f_\epsilon(\alpha_0, \alpha_1, \alpha_2) = \sum_{i=1}^{n} (\alpha_0 + \alpha_1 * x_i + \alpha_2 * x_i^2 - y_i)^2
$$

Auch für diese Residuenfunktion lassen sich alle partiellen Ableitungen bilden:

$$
\frac{\partial}{\partial \alpha_0} f_{\epsilon}(\alpha_0, \alpha_1, \alpha_2) = 2 * \sum_{i=1}^{n} \alpha_0 + \alpha_1 x_i + \alpha_2 x_i^2 - y_i
$$
$$
\frac{\partial}{\partial \alpha_1} f_{\epsilon}(\alpha_0, \alpha_1, \alpha_2) = 2 * \sum_{i=1}^{n} x_i * (\alpha_0 + \alpha_1 x_i + \alpha_2 x_i^2 - y_i)
$$
$$
\frac{\partial}{\partial \alpha_2} f_{\epsilon}(\alpha_0, \alpha_1, \alpha_2) = 2 * \sum_{i=1}^{n} x_i^2 * (\alpha_0 + \alpha_1 x_i + \alpha_2 x_i^2 - y_i)
$$

Durch die Suche nach Nullstellen ergibt sich das folgende Gleichungssystem:

$$
\sum_{i=1}^{n} y_i = \alpha_0 * n + \alpha_1 * \sum_{i=1}^{n} x_i + \alpha_2 * \sum_{i=1}^{n} x_i^2
$$
$$
\sum_{i=1}^{n} x_i y_i = \alpha_0 * \sum_{i=1}^{n} x_i + \alpha_1 * \sum_{i=1}^{n} x_i^2 + \alpha_2 * \sum_{i=1}^{n} x_i^3
$$
$$
\sum_{i=1}^{n} x_i^2 y_i = \alpha_0 * \sum_{i=1}^{n} x_i^2 + \alpha_1 * \sum_{i=1}^{n} x_i^3 + \alpha_2 * \sum_{i=1}^{n} x_i^4
$$

Diese drei Gleichungen lassen sich am einfachsten notieren, wenn sie als lineares Gleichungssystem der Form $A * x = b$ notiert werden.

$$
\begin{pmatrix}
	n & \sum_{i=1}^{n} x_i & \sum_{i=1}^{n} x_i^2 \\
	\sum_{i=1}^{n} x_i & \sum_{i=1}^{n} x_i^2 & \sum_{i=1}^{n} x_i^3 \\
	\sum_{i=1}^{n} x_i^2 & \sum_{i=1}^{n} x_i^3 & \sum_{i=1}^{n} x_i^4
\end{pmatrix} * \begin{pmatrix}
	\alpha_0 \\
	\alpha_1 \\
	\alpha_2
\end{pmatrix} = \begin{pmatrix}
	\sum_{i=1}^{n} y_i \\
	\sum_{i=1}^{n} x_i y_i \\
	\sum_{i=1}^{n} x_i^2 y_i
\end{pmatrix}
$$

Eine Umsetzung in Python könnte nun wie folgt aussehen. Zunächst wird die Matrix $A$ gebildet, indem abhängig von Reihe und Spalte die entsprechenden Potenzen aller $x$-Koordinaten gebildet und aufsummiert werden. Analog geschieht dies für den Vektor $b$. Die Funktion `solve` aus Numpy berechnet dann den Lösungsvektor $x$, der die Werte für $a_0$, $a_1$ und $a_2$ enthält.

In [ ]:
A = np.zeros((3, 3))
for row in range(3):
    for col in range(3):
        A[row,col] = (df_sqr['x'] ** (row+col)).sum()

b = np.zeros(3)
for row in range(3):
    b[row] = ((df_sqr['x'] ** row) * df_sqr['y']).sum()

x = np.linalg.solve(A, b)  # [a_0, a_1, a_2]
x

An den Parametern lässt sich bereits erkennen, dass sie sehr nah an den ursprünglich gewählten Koeffizienten liegen. In der grafischen Darstellung bestätigt sich dieser Eindruck erneut.

In [ ]:
fig = px.scatter(df_sqr, x='x', y='y')
draw_regression(fig, *x)

Sofern der Polynomgrad bekannt ist, lassen sich wie bereits erwähnt auch noch deutlich komplexere Polynome durch eine Ausgleichskurve beschreiben. Die folgenden zwei Zellen zeigen beispielhaft die Anpassung an ein Polynom vierten Grades.

In [ ]:
np.random.seed(1)

df_quar = pd.DataFrame({
    'x': (random.random() * 8 - 3.75 for _ in range(200))
})
df_quar['y'] = df_quar['x'] ** 4 - 1 * df_quar['x'] ** 3 - 14 * df_quar['x'] ** 2 + 5 * df_quar['x']
df_quar['y'] += np.random.normal(loc=0, scale=3, size=200)

px.scatter(df_quar, x='x', y='y', trendline='ols')

Beachten Sie, dass die Größe der Matrix $A$ und des Vektors $b$ angepasst werden muss.

In [ ]:
A = np.zeros((5, 5))
for row in range(5):
    for col in range(5):
        A[row,col] = (df_quar['x'] ** (row+col)).sum()

b = np.zeros(5)
for row in range(5):
    b[row] = ((df_quar['x'] ** row) * df_quar['y']).sum()

x = np.linalg.solve(A, b)  # [a_0, a_1, a_2, a_3, a_4]

fig = px.scatter(df_quar, x='x', y='y')
draw_regression(fig, *x)

## Höherdimensionale Funktionen
Laden Sie zuerst den folgenden Datensatz, lassen Sie ihn grafisch darstellen und suchen Sie visuell nach einer Ausgleichsebene.

In [ ]:
df_hdf = pd.read_csv(xyz_regression_path)
go.Figure(data=[go.Scatter3d(x=df_hdf['x'], y=df_hdf['y'], z=df_hdf['z'], mode='markers')])

Die Ausgleichsebene sei nun gegeben durch die nachfolgende Gleichung:
$$
z = \alpha_0 + \alpha_1 x + \alpha_2 y
$$

Analog zur Behandlung höhergradiger Polynome lassen sich nun die Residuen sowie die Residuenfunktion formulieren. Mit Hilfe der partiellen Ableitungen lässt sich nun ein Extremum suchen, indem das entstehende Gleichungssystem mit der bereits zuvor angewandten Funktion `solve` gelöst wird.